# SAM2 + LaMa: Simple Furniture Background Removal

This simplified notebook uses SAM2 for segmentation and LaMa for inpainting to remove backgrounds from furniture images.

In [ ]:
# @title 1️⃣ Quick Install { display-mode: "form" }
# @markdown One-click installation - just run and wait

!pip install -q segment-anything-2 simple-lama-inpainting
!pip install -q opencv-python matplotlib numpy pillow scipy

print("✅ Installation complete!")

In [ ]:
# @title 2️⃣ Import and Initialize { display-mode: "form" }
# @markdown Load all required libraries

import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import files
from scipy import ndimage

# Import SAM2 and LaMa
from sam2.sam2_image_predictor import SAM2ImagePredictor
from simple_lama_inpainting import SimpleLama

# Initialize models
print("Loading models...")
predictor = SAM2ImagePredictor.from_pretrained("facebook/sam2-hiera-small")
lama = SimpleLama()

print("✅ Models loaded!")

In [ ]:
# @title 3️⃣ Upload Image { display-mode: "form" }
# @markdown Upload your furniture image

print("Upload your furniture image:")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Load and display image
image = Image.open(filename).convert("RGB")
image_np = np.array(image)

plt.figure(figsize=(10, 8))
plt.imshow(image)
plt.title(f"Original Image ({image.size[0]}x{image.size[1]})")
plt.axis('off')
plt.show()

# Set image for SAM2
predictor.set_image(image_np)

In [ ]:
# @title 4️⃣ Select Furniture { display-mode: "form" }
# @markdown Click on the furniture center point

# Get center point as default
center_x = image.size[0] // 2
center_y = image.size[1] // 2

print("Enter coordinates (or press Enter for center):")
x = int(input(f"X coordinate (default {center_x}): ") or center_x)
y = int(input(f"Y coordinate (default {center_y}): ") or center_y)

# Generate mask
masks, scores, _ = predictor.predict(
    point_coords=np.array([[x, y]]),
    point_labels=np.array([1]),
    multimask_output=False
)

mask = (masks[0] > 0.5).astype(np.uint8) * 255

# Show mask
plt.figure(figsize=(10, 8))
plt.imshow(mask, cmap='gray')
plt.title("SAM2 Mask")
plt.axis('off')
plt.show()

In [ ]:
# @title 5️⃣ Smart Background Removal { display-mode: "form" }
# @markdown Remove background and fix missing parts

# Fill holes in mask (for missing cushions/parts)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (50, 50))
closed_mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
filled_mask = ndimage.binary_fill_holes(closed_mask).astype(np.uint8) * 255

# Find missing parts to inpaint
missing_parts = cv2.bitwise_and(filled_mask, cv2.bitwise_not(mask))

# Create background-removed image
bg_removed = image.copy()
bg_array = np.array(bg_removed)
bg_array[mask == 0] = 255  # White background
bg_removed = Image.fromarray(bg_array)

# Detect white artifacts inside furniture
gray = cv2.cvtColor(bg_array, cv2.COLOR_RGB2GRAY)
white_artifacts = ((gray > 240) & (filled_mask > 0)).astype(np.uint8) * 255

# Combine areas to inpaint
inpaint_mask = cv2.bitwise_or(missing_parts, white_artifacts)
kernel_small = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
inpaint_mask = cv2.dilate(inpaint_mask, kernel_small, iterations=1)

# Run LaMa inpainting
print("Running inpainting...")
if inpaint_mask.max() > 0:  # Only inpaint if there are areas to fix
    result = lama(image, Image.fromarray(inpaint_mask))
else:
    result = image

# Apply final mask
final = Image.new('RGB', result.size, (255, 255, 255))
result_array = np.array(result)
final_array = np.array(final)
final_array[filled_mask > 0] = result_array[filled_mask > 0]
final = Image.fromarray(final_array)

# Show results
fig, axes = plt.subplots(1, 3, figsize=(20, 8))
axes[0].imshow(image)
axes[0].set_title("Original")
axes[0].axis('off')

axes[1].imshow(bg_removed)
axes[1].set_title("SAM2 Result")
axes[1].axis('off')

axes[2].imshow(final)
axes[2].set_title("Final Result")
axes[2].axis('off')
plt.show()

print("✅ Background removal complete!")

In [ ]:
# @title 6️⃣ Save Results { display-mode: "form" }
# @markdown Download the final image

# Save and download
output_name = filename.rsplit('.', 1)[0] + "_no_bg.png"
final.save(output_name)
files.download(output_name)

print(f"✅ Saved as {output_name}")